In [ ]:
import requests
import pandas as pd

# --- 1. Récupérer toutes les courses terminées 2025 pour historique ---

url_results = "https://api.jolpi.ca/ergast/f1/2025/results/"
response = requests.get(url_results)
data = response.json()

races = data['MRData']['RaceTable']['Races']

results = []
for race in races:
    # Filtrer uniquement courses terminées (Status == Finished)
    for result in race['Results']:
        if result['status'] == 'Finished':
            results.append({
                'Round': int(race['round']),
                'Race': race['raceName'],
                'Date': race['date'],
                'Driver': result['Driver']['familyName'],
                'Constructor': result['Constructor']['name'],
                'Grid': int(result['grid']),
                'Position': int(result['position']),
                'Status': result['status'],
                'Points': float(result['points'])
            })

df_history = pd.DataFrame(results)

# Calculs historiques par pilote
points = df_history.groupby('Driver')['Points'].sum()
victoires = df_history[df_history['Position'] == 1].groupby('Driver').size()
position_moy = df_history.groupby('Driver')['Position'].mean()

df_history_stats = pd.DataFrame({
    'Points': points,
    'Victoires': victoires,
    'Position_moy': position_moy
}).fillna(0)

# --- 2. Récupérer la liste des courses 2025 pour trouver le round souhaité ---

races_url = "https://api.jolpi.ca/ergast/f1/2025/races/"
races_response = requests.get(races_url)
races_data = races_response.json()
races_list = races_data['MRData']['RaceTable']['Races']

race_name = 'Monaco Grand Prix'  # Modifier selon la course voulue
selected_race = next((race for race in races_list if race['raceName'] == race_name), None)
if selected_race is None:
    raise ValueError(f"Course '{race_name}' non trouvée")

round_number = selected_race['round']

# --- 3. Récupérer les résultats de qualification de cette course ---

qualifying_url = f"https://api.jolpi.ca/ergast/f1/2025/{round_number}/qualifying/"
qualifying_response = requests.get(qualifying_url)
qualifying_data = qualifying_response.json()
qualifying_results = qualifying_data['MRData']['RaceTable']['Races'][0]['QualifyingResults']

qualifying_info = []
for q in qualifying_results:
    qualifying_info.append({
        'Driver': q['Driver']['familyName'],
        'QualifyingPosition': int(q['position'])
    })

df_qualifying = pd.DataFrame(qualifying_info)

# --- 4. Fusionner historique et qualifications ---

df = df_history_stats.merge(df_qualifying, on='Driver', how='inner')

# --- 5. Calcul des scores et normalisation ---

df['InvQualiPos'] = 1 / df['QualifyingPosition']
df['InvPosMoy'] = 1 / df['Position_moy']

# Normalisation
df['Points_norm'] = df['Points'] / df['Points'].max()
df['Victoires_norm'] = df['Victoires'] / df['Victoires'].max()
df['InvPosMoy_norm'] = df['InvPosMoy'] / df['InvPosMoy'].max()
df['InvQualiPos_norm'] = df['InvQualiPos'] / df['InvQualiPos'].max()

# Score final (vous pouvez ajuster les poids ici)
df['PronosticScore'] = (
    df['Points_norm'] * 0.4 +
    df['Victoires_norm'] * 0.2 +
    df['InvPosMoy_norm'] * 0.2 +
    df['InvQualiPos_norm'] * 0.2
)

# Tri décroissant selon le score
df = df.sort_values(by='PronosticScore', ascending=False)

print(df[['Driver', 'PronosticScore']])